# bilibili PCA UserCF

企图将番剧作为维度，进行主成分分析，之后降维，再进行聚类得到不同的人群

In [2]:
# import package
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd

In [3]:
AnimeDic = pd.read_json("D:/ML/animeId.json")[0].tolist()
def AnimeIdConvert(AnimeId):
    return AnimeDic.index(int(AnimeId))
#load rawdata
with open('D:/ML/alluserlocal.json', 'r',encoding="utf-8") as f:    
    # 逐行读取    
    data = (line.strip() for line in f)    
    # 格式化，然后每行作为list的一个元素    
    data_json = "[{0}]".format(','.join(data))
    recipes = pd.read_json(data_json)
raw_data = recipes["likevideo"].tolist()
tmp_data = []
for d in raw_data:
    if len(d) > 10:
        tmp_data.append(d)
raw_data = tmp_data
# change to shape(x, 3402)
data = []
for d in raw_data:
    arr = np.zeros(3402)
    for bungmi in d:
        try:
            arr[AnimeIdConvert(bungmi)] = 1
        except ValueError:
            continue
    data.append(arr)

In [4]:
# pca test
pca = PCA(n_components=30)
newd = pca.fit_transform(data)

使用聚类的效果非常的差，我们最终决定是比较新向量和每一个向量求取一个余弦相似度

In [6]:
从上面的结果初步来看是比较正确的，但为了节省时间我们需要更加快速的找到我们的相似用户，且两个用户的相似度不能过于高，否则很能实现长尾，推荐的意义并不大，并且我们需要进行二次封装，封装为与原来的接口相似，为了达到快速寻找和泛化的效果，我准备进行多层次聚类达到类似树状结构的查找，我准备弄成json数据存在mongoDB里面

In [33]:
# k-means
from sklearn.cluster import KMeans
TurnOne = KMeans(n_clusters=20, random_state=0)
TurnOneResult = TurnOne.fit_predict(newd)
TurnOneCenters = TurnOne.cluster_centers_ # 得到第一层的中心

In [67]:
def getTurnData(TurnResult):
    TurnData = []
    for index in range(20):
        pos_array = np.argwhere(TurnResult == index)
        TurnUser = []
        for pos in pos_array:
            TurnUser.append(newd[pos][0])
        TurnData.append(TurnUser)
    return TurnData
TurnOneData = getTurnData(TurnOneResult)# 得到第一层的分类结果
TurnTwoTotal = []
TurnTwoCenters = []
for label in TurnOneData:
    TurnTwo = KMeans(n_clusters=20, random_state=0)
    TurnTwoResult = TurnTwo.fit_predict(label)
    TurnTwoData = getTurnData(TurnTwoResult)
    TurnTwoTotal.append(TurnTwoData)
    TurnTwoCenters.append(TurnTwo.cluster_centers_)

(20,)


In [74]:
# TurnOneCenters and TurnTwoCenters are like sifters
np.array(TurnTwoTotal).shape
#np.array(TurnTwoCenters).shape

(20,)

In [77]:
from sklearn.metrics.pairwise import cosine_similarity
from random import shuffle
def ToAnimeId(index):
    return AnimeDic[index]
def predict(my_favorite):
    convert_favorite = np.zeros(3402)
    for ban in my_favorite:
        convert_favorite[AnimeIdConvert(ban)] = 1
    convert_favorite = pca.transform([convert_favorite])
    max1_index = 0
    index = 0
    max_similar = 0.0
    for user in TurnOneCenters:
        similar = cosine_similarity([user],convert_favorite)
        if similar > max_similar:
            max_similar = similar
            max1_index = index
        index += 1
    # 已在TurnOne找到最相似的中心
    Turn2 = TurnTwoCenters[max1_index]
    max2_index = 0
    index = 0
    max_similar = 0.0
    for center in Turn2:
        similar = cosine_similarity([user],convert_favorite)
        if similar > max_similar:
            max_similar = similar
            max2_index = index
        index += 1
    # 已在TurnTwo找到最相似的中心
    friends = TurnTwoTotal[max1_index][max2_index]
    # 随机从friends中抽取一个作为推荐的比对
    count = 0
    recommend_list = []
    while(count < 10):
        shuffle(friends)
        simi = pca.inverse_transform(friends[0])
        order = np.argsort(simi)
        order = order[::-1]
        for i in range(5):
            if ToAnimeId(order[i]) in my_favorite or ToAnimeId(order[i]) in recommend_list:
                continue
            else:
                recommend_list.append(ToAnimeId(order[i]))
                count += 1
    return recommend_list[:10]

In [78]:
my_favorite=[2661,645,5997,1547,2747,1293,5800,5798,1057,3048,5027,5793,6440,21557,963,427,177,3756]# test data
predict(my_favorite)

[1656, 1539, 836, 1699, 1505, 6402, 5852, 2543, 6301, 3450]

In [86]:
# for more convinient, we transform the id to bangumi's name
bilibili = {}
import json
with open('D:/ML/bilibili.json', 'r',encoding="utf-8") as f:    
    # 逐行读取    
    for line in f:
        bangumi = json.loads(line)
        # 格式化，然后每行作为list的一个元素    
        bilibili[bangumi["animeId"]] = bangumi["animeTitle"]
def Id2Name(commend_list):
    name_list = []
    for commend in commend_list:
        name_list.append(bilibili[str(commend)])
    return name_list

In [89]:
Id2Name(predict(my_favorite))

['Re：从零开始的异世界生活',
 '欢迎来到实力至上主义的教室',
 '徒然喜欢你',
 '干物妹！小埋R',
 'Fate/stay night [Unlimited Blade Works] 第一季',
 '魔卡少女樱 CLEAR CARD篇',
 'Fate/stay night [Unlimited Blade Works] 第二季',
 '全职高手 第一季',
 '紫罗兰永恒花园',
 '干物妹！小埋']

In [88]:
Id2Name(my_favorite)

['我的妹妹不可能那么可爱 第二季',
 '我的妹妹是大阪大妈',
 '埃罗芒阿老师',
 '吹响吧！上低音号',
 '终物语',
 '幸运星',
 '小林家的龙女仆',
 'One Room',
 '凉宫春日的忧郁 2009版',
 'CROSSANGE 天使与龙的轮舞',
 'NEW GAME!',
 '珈百璃的堕落',
 '如果有妹妹就好了。',
 'citrus～柑橘味香气～',
 '魔法禁书目录 第一季',
 '某科学的超电磁炮S',
 '漫画家与助手',
 '魔卡少女樱']